This code module picks up where the 00 module left off.  Data has been downloaded and the raw_data.csv file has been renamed to raw_data_lhbvlh_2014_2015.csv.

We can check by looking at the directory Output/raw_data subdirectory. If you ran the code in 00 module, the file will be named raw_data.csv. the file posted to gitub and included in the example utility in the container is named raw_data_lhbvlh_2014_2015.csv.  First we'll load required packages and then check the raw_data directory to make sure the file we want is there.


In [ ]:
import os
import pandas as pd
import pandas.io.sql as pdsqlio
import psycopg2
import getpass
import re

try:
    os.chdir('Output/raw_data')
except:
    print('no such directory')
print(os.getcwd())


! ls -lta # bash shell directory command


We have data to import, so next we will connect to the database and do some setup work. We begin by building a connection to the postgres database.


In [ ]:
db=input('enter dbname: ')
user=input('enter user: ')
print('enter password: ')
password=getpass.getpass()
conn_str="host='localhost' dbname='"+db+"' user='"+user+"' password='"+password+"' options='-c search_path=sandbox,public'"
# the options allows us to add a schema called sandbox and be able to work in it
conn=psycopg2.connect(conn_str)
cur=conn.cursor()
cur.execute('select version()')
print(cur.fetchone()[0])

We are connected to the database.  For importing raw data that will be manipulated into a final format, lets put it into a sandbox schema to keep it separate from our 'good' data.


In [ ]:
str_sql = "SELECT exists(select schema_name FROM information_schema.schemata WHERE schema_name = 'sandbox');"
cur.execute(str_sql)
conn.commit()
schema_exists =cur.fetchone()[0]
print(schema_exists)
if schema_exists==False:
    str_sql="create schema sandbox;"
    cur.execute(str_sql)
    conn.commit()
str_sql='select schema_name FROM information_schema.schemata;'
cur.execute(str_sql)
conn.commit()
print(cur.fetchall())


Now that we have created or have confirmed the existance of the sandbox schema, we should be able to use it.


In [ ]:
str_sql = "SELECT exists(select * FROM information_schema.tables WHERE table_schema = 'sandbox' and table_name = 'test');"
cur.execute(str_sql)
conn.commit()
table_exists =cur.fetchone()[0]
print(table_exists)
str_sql_drop = "drop table sandbox.test;"
if table_exists==True:
    cur.execute(str_sql_drop)

str_sql = "create table sandbox.test(name varchar(100), primary key(name));"
cur.execute(str_sql)
str_sql = "insert into sandbox.test(name) values('George');"
cur.execute(str_sql)
str_sql = "insert into sandbox.test(name) values('Jane');"
cur.execute(str_sql)
str_sql = "insert into sandbox.test(name) values('Judy');"
cur.execute(str_sql)
str_sql = "insert into sandbox.test(name) values('Elroy');"
cur.execute(str_sql)
str_sql = "insert into sandbox.test(name) values('Astro');"
cur.execute(str_sql)
str_sql = "insert into sandbox.test(name) values('Rosie');"
cur.execute(str_sql)
conn.commit()

str_sql = "select * from sandbox.test;"
cur.execute(str_sql)
conn.commit()
print(cur.fetchall())


Now that we've tested our connection string and can add tables to the sandbox schema, we can detlete the test table then pull in the MERRA2 data.


In [ ]:
cur.execute(str_sql_drop)
conn.commit()

There are cases where the file might be larger than the computer memory running this notebook. We will get just the first few rows to determine the file format.

In [ ]:
fname='raw_data_lhbvlh_2014_2015.csv'
if os.path.isfile(fname):
    data = pd.read_csv(fname, nrows=50)
else:
    raise SystemExit("Stop right there!")
    
x=data.dtypes
print(x)

As noted in the 00 file, the MERRA2 data is fully described in https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf. 

We can use the dtypes to create a table in our postgres database. 


In [ ]:
print(fname)

In [ ]:
# note this is strictly a drop and replace, it will not append data as written
def maketable(fname):
    type_map={'float64':'real',
          'int64':'bigint',
          'object':'timestamp'}
    newtablename=re.sub('.csv','',fname,flags=re.I)
    cur.execute("SELECT EXISTS(SELECT * FROM information_schema.tables "+
                "WHERE table_schema = 'sandbox' AND table_name = '"+newtablename+"');")
    if cur.fetchone()[0]:
        cur.execute("DROP TABLE "+newtablename+';')
        conn.commit()
        msg='deleted and replaced'
    else:
        msg='created new'
    data = pd.read_csv(fname,nrows=50)
    types=data.dtypes
    str_sql='CREATE TABLE '+newtablename+' ('
    len_x=len(data.columns.tolist())
    for x in data.columns.tolist():
        str_sql=str_sql+x+' '+  type_map[str(types[x])]
        if(x!=data.columns.tolist()[len_x-1]):
            str_sql=str_sql + ', '
        else :
            str_sql=str_sql+ ');'
    cur.execute(str_sql)
    conn.commit()
    return [msg,str_sql]  

maketable(fname)


The table has been created based on the structure of the data, so no we can import the csv directly into the database.


In [ ]:
tablename=re.sub('.csv','',fname,flags=re.I)
f=open(fname)
cur.copy_expert("copy sandbox."+tablename+" from STDOUT delimiter ',' csv header",f)
conn.commit()
cur.close()

Now that the data is in the database, we can pull what we want into pandas, or continue working with data in teh database and doing data wrangling and munging directly with sql.

Here's an example of pullin data into pandas using its ability to read sql.


In [ ]:
str_sql = "select * from sandbox."+tablename+" limit 5;"
data=pdsqlio.read_sql_query(str_sql,conn)
data